In [ ]:
# -*- coding: utf-8 -*-
import urllib2
import mechanize
import BeautifulSoup
import codecs
import csv,xlrd,json
from urlparse import urlparse
from pprint import pprint
import gspread,os
from oauth2client.client import SignedJwtAssertionCredentials
import json
import os
import time
import requests
from PIL import Image
from StringIO import StringIO
from requests.exceptions import ConnectionError
import urllib
import readPicFileAsDictionary as picDic
from bingSearch import BingSearch
import sqlite3

def go(query,lang):
    """Download full size images from Google image search.
    Don't print or republish images without permission.
    I used this to train a learning algorithm.
    """
    
    """https://developers.google.com/image-search/v1/jsondevguide#json_args"""
    
    BASE_URL = 'https://ajax.googleapis.com/ajax/services/search/images?'\
             'v=1.0&q=' + query + '&start=%d&as_rights=cc_publicdomain'

    urls = []
    start = 0 # Google's start query string parameter for pagination.
    while start < 60: # Google will only return a max of 56 results.
        r = requests.get(BASE_URL %start)
        print (BASE_URL %start)
        
        js = json.loads(r.text)
        if(js["responseStatus"] != 200):
            print js["responseDetails"]
        if r and js:
            resData = js['responseData']
            if resData and resData['results']:
                for image_info in resData['results']:
                    url = image_info['unescapedUrl']
                    if "pixabay" not in url and "fotocommunity" not in url and ".svg" not in url and ".gif" not in url:                        
                        if len(urls) > 9:
                            break
                        print url
                        urls.append(url)
                        
            start += 4 # 4 images per page.
        if len(urls) > 10:
            break
        # Be nice to Google and they'll be nice back :)
        time.sleep(10)
        
    return urls
    

def getPics(searchURL):
    img_urls = []
    browser = mechanize.Browser()
    browser.set_handle_robots(False)
    browser.addheaders = [('User-agent',
                           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.114 Safari/537.36')]
    browser.set_proxies({"http": "10.200.1.26:8080", "https": "10.200.1.26:8080"})
    try:
        print searchURL.encode('utf-8')
        res = browser.open(searchURL.encode('utf-8'))
        htmltext = res.read()
        soup = BeautifulSoup.BeautifulSoup(htmltext)
        results = soup.findAll("a")
        count = 0
        for r in results:
            if r.get('href') != None:
                hrf = r.get('href')
                if ("imgres?imgurl" in hrf and "pixabay" not in hrf and "fotocommunity" not in hrf and ".svg" not in hrf and ".gif" not in hrf):

                    s = hrf
                    if("&imgrefurl" in hrf):
                        fs = s[s.find("imgres?imgurl="):s.find("&imgrefurl")]
                    else:
                        fs = s[s.find("imgres?imgurl="):]
                        print hrf

                    print fs
                    fs = fs.replace("imgres?imgurl=", "")
                    # print fs
                    fs = urllib2.unquote(fs)
                    fs = urllib2.unquote(fs)

                    if count < 20:
                        # print fs
                        img_urls.append(fs)
                        count += 1
                    else:
                        return img_urls

        return img_urls

    except Exception, arg:
        print "Error" + str(arg)
        if not len(img_urls) == 0:
            return img_urls
        else:
            return None


def alfa_unicode_dict_reader(fName, fencoding):

    with open(fName) as mfile:
        # mfile = open(fName,mode="rb")
        hd = mfile.readline()
        values = mfile.readlines()
        # hd = values[0]
        keys = hd.strip().split(",")
        output = []
        for row in values:
            row = unicode(row, fencoding)
            result = dict(zip(keys, row.strip().split(',')))
            output.append(result)
    return output


def readAndWriteFile(input_file):
    cf = alfa_unicode_dict_reader(input_file, "utf-8")
    fout = codecs.open("ForMechTurkImgSel-g-fr.csv", "wb", encoding='utf-8')
    fout.write("word,category,img1,img2,img3,img4,img5,img6,img7,img8,img9,img10\n")
    for rw in cf:
        if not (rw['word'] == "" or rw['word'] == None):
            search = rw['word']
            print "searching : " + search
            # search = search + "+" + rw['Input.category']
            search = search.replace(" ", "+")
            types = "Any"
            lang = "fr"
            url = "https://www.google.co.in/search?hl="+lang+"&authuser=0&site=imghp&tbm=isch&tbs=itp:" + types + ",isz:m,sur:fc&q=" + search
            # print url
            reA = getPics(url)
            finStr = []
            finStr.append(rw['word'])
            finStr.append(rw['Input.category'])
            if (reA != None and len(reA) != 0):
                for itm in reA:
                    finStr.append(itm)

            urStr = ",".join([("\"" + i + "\"") for i in finStr])
            fout.write(urStr)
            fout.write("\n")

    fout.close()


def escape_comma(word):
    return word.replace(",","%2C")



def get_file_with_header(file_name):
    fl = codecs.open(file_name, "wb", encoding='utf-8')
    fl.write("<html>")
    fl.write("<meta http-equiv=\"Content-Type\" charset=\"UTF-8\" />")
    fl.write("<link rel=\"stylesheet\" type=\"text/css\" href=\"./css/alfastyle.css\">")
    fl.write("<head><script src=\"http://ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.min.js\"></script><script src=\"./js/jquery.cookie.js\"></script><script src=\"./js/alfa.js\"></script></head>")
    fl.write("<Body>")
    fl.write("<Form name=\"input\" action = \"\">")
    return fl




def read_write2html(input_file):
    oput = codecs.open("oput.csv", 'w', encoding='utf8')

    fl = get_file_with_header("results-g-fr-1.html")

    counter = 0
    file_counter = 1

    cf = alfa_unicode_dict_reader(input_file, "utf-8")
    nmatch = 0

    # print cf
    for i in range(len(cf)):
        rw = cf[i]
        if not (rw['word'] == "" or rw['word'] == None):
            search = rw['word']
            cati = rw['Input.category']
            # print search
            srch = "<h2>" + search + " - " + rw['Input.category'] + "</h2>"
            print "searching : " + search
            gsearch = search.strip("\"").strip()
            # gsearch = gsearch + "+" + cati
            gsearch = gsearch.replace(" ", "+")
            search = search.replace(" ", "+")
            types = "Any"
            lang = "fr"
            url = "https://www.google.fr/search?hl="+lang+"&authuser=0&site=imghp&tbm=isch&safe=active&tbs=itp:" + types + ",isz:m,sur:fc&q=" + gsearch
            # print url
            fl.write(srch)
            # gSrh = "<a class = \"alinky\" href = \"" + url + "\">Search In Google</a></br>"
            fl.write("<div class =\"wrappy\">")
            # fl.write(gSrh)
            reA = getPics(url)
            # print reA
            if (reA != None and len(reA) != 0):
                for itm in reA:
                    fl.write("<div class =\"gridy\">")
                    # print type(itm)
                    itg = "<a href=\"" + itm + "\">" + "<img src = \"" + itm + "\" class =\"imagy\" default = \"img\"></img></a>"

                    fl.write(itg)
                    fl.write("</br>")
                    checkbox = rw['word'].strip("\"").strip() + "," + rw['Input.category'].strip(
                        "\"").strip() + "," + "'" + itm + "'"
                    chbox = "<input class =\"checky\" type=\"checkbox\" name=\"" + checkbox + "\">"
                    # print chbox
                    fl.write(chbox)
                    oput.write(checkbox + "\n")
                    fl.write("</div>")

            fl.write("<div class =\"gridy\">")
            nmatch += 1
            fl.write("<h3>No Match</h3>")
            checkbox = rw['word'].strip("\"").strip() + "," + rw['Input.category'].strip("\"").strip() + "," + "NO-MATCH-"+str(nmatch)
            fl.write("<input class =\"checky\" type=\"checkbox\" name=\"" + checkbox + "\">")
            fl.write("</div>")
            fl.write("</div>")
            fl.write("</br>")
            counter += 1

        if counter == 10 or len(cf)-i<10:
            print "****************************"
            fl.write("<input type=\"submit\" value=\"Submit\">")
            fl.write("</Form>")
            fl.write("</Body>")
            fl.write("</html>")
            fl.close()
            file_counter += 1
            fnme = "results-g-fr-" + str(file_counter) + ".html"
            fl = get_file_with_header(fnme)
            counter = 0
    oput.close()
 
 
def bing_search(query):
    search_type = 'Image'
    #search_type: Web, Image, News, Video
    key= 'FM1eRnjFYoltkKdRDKcwiJMk9VFn9p/yRgru20R5xc4'
    query = urllib.quote(query)
    # create credential for authentication
    user_agent = 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; FDM; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 1.1.4322)'
    credentials = (':%s' % key).encode('base64')[:-1]
    auth = 'Basic %s' % credentials
    url = 'https://api.datamarket.azure.com/Data.ashx/Bing/Search/'+search_type+'?Query=%27'+query+'%27&$top=50&$format=json'
    request = urllib2.Request(url)
    request.add_header('Authorization', auth)
    request.add_header('User-Agent', user_agent)
    request_opener = urllib2.build_opener()
    response = request_opener.open(request) 
    response_data = response.read()
    json_result = json.loads(response_data)
    result_list = json_result['d']['results']
    resArray = []
    for res in result_list:
        url = res[u'MediaUrl']
        if ("wikimedia" in url) and ("commons" in url):
            if len(resArray) > 60:
                break
            resArray.append(url)
            print url
    return resArray



def start(input_file,outpath,lang,picFilePath):
    """
    Args:
    input_file: The first parameter.
    outpath: The second parameter.
    lang: The third parameter.
    picFilePath: The fourth parameter.
    """
    data = readXL(input_file)
    symbol1,symbol2 = picDic.readSym_Stix(lang,picFilePath)
    print len(data)
#     print data
    outpath = os.path.join(outpath,"jsons/")
    print outpath
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    print outpath
    fout = codecs.open(os.path.join(outpath , "json-1.json"), "wb", encoding='utf-8')
    counter = 0
    file_counter = 1
    nmatch = 0

    fout.write("[")
    bing = BingSearch('FM1eRnjFYoltkKdRDKcwiJMk9VFn9p/yRgru20R5xc4')
    print bing
    for i,dd in enumerate(data):
        rw={}
        rw['Input.category']=dd[0]
        rw['word']=dd[1]    
        
        if not (rw['word'] == "" or rw['word'] == None):
            search = rw['word'].strip("\"").strip()
            cati = rw['Input.category'].strip("\"").strip("'").strip()
            print "searching : " + search
            types = "Any"
            reA = ""
            if search in symbol1:
                reA = [os.path.join("png",x) for x in symbol1[search]]
            elif search in symbol2:
                reA = [os.path.join("png",x) for x in symbol2[search]]
            else:                
                reA = bing.bing_search(search.encode('utf-8'),"Image")

            if (reA != None and len(reA) != 0):
                lst = 0
                counter += 1
                fout.write("{\"" + search + " - " + cati + "\":[")
                for itm in reA:
                    lst += 1
                    ckbx = search + "," + cati + ",'" + itm + "'"
                    
                    fout.write("{\"imgURL\":\"" + itm + "\",\"checkName\":\"" + ckbx + "\"}")
                    fout.write(",")

                nmatch += 1
                ckbx = search + "," + cati + "," + "No-Match-" + str(nmatch)
                fout.write("{\"imgURL\":\"http://upload.wikimedia.org/wikipedia/commons/7/75/Erroricon404.PNG\",\"checkName\":\"" + ckbx + "\"}")
                fout.write("]}")
                if not counter > 9:
                    fout.write(",")
#             else:
#                 nmatch += 1
#                 ckbx = search + "," + cati + "," + "No-Match-" + str(nmatch)
#                 fout.write("{\"imgURL\":\"http://upload.wikimedia.org/wikipedia/commons/7/75/Erroricon404.PNG\",\"checkName\":\"" + ckbx + "\"}")

        
            


        if counter == 10 or len(data)-i == 1:
            print counter
            fout.write("]")
            file_counter += 1
            fout.close()
            if not len(data)-i == 1:
                fname = os.path.join(outpath,"json-" + str(file_counter) + ".json")
                fout = codecs.open(fname, "wb", encoding='utf-8')
                fout.write("[")
                counter = 0
#         else:
#             fout.write(",")



def readXL(path):
    global sqlPath
    con = sqlite3.connect(sqlPath)
    cur = con.cursor()
    bk = xlrd.open_workbook(path)
    sht = bk.sheet_by_name("Localization")
    data=[]
    for row in range(1,sht.nrows):
        action = sht.cell_value(row,0).strip().lower()
        avazCat = sht.cell_value(row,1).strip().upper()
        avazCatofCat = sht.cell_value(row,2).strip().upper()
        swedCat = sht.cell_value(row,3).strip().upper()
        swedCatofCat = sht.cell_value(row,4).strip().upper()
        itemsStr = sht.cell_value(row,6).strip()
        items = [x.strip() for x in itemsStr.split(",")]
        outPut = codecs.open("/Users/alfred/Downloads/imageSrch.csv", 'w', encoding='utf8')
        for item in items:
            dbData = cur.execute("select * from zpicmodedict where ztag_name=?",(item,)).fetchall()
            if dbData:
                continue
            if item:
                item = item.replace('"',"'")
                if avazCat:
                    data.append((avazCat,item))
                elif swedCat:
                    data.append((swedCat,item))
    print "Count:",len(data)
    return data
            
   

# inputXLFile = "/Users/alfredreynold/Downloads/Avaz på svenska - Avaz Swedish (2).xlsx"
# jsonOutputFolder = "/Users/alfredreynold/ALFRED/DEV/snakes/Translation/SwedishJson"
# picFilePath = "/Users/alfredreynold/ALFRED/DEV/Avaz-content-tools/symbols/picFile-latest.xlsx"
# sqlPath = "/Users/alfredreynold/ALFRED/DEV/snakes/Translation/User-Data.sqlite"
# start(inputXLFile,jsonOutputFolder,"sv",picFilePath)
# bing_search("coffee")
# go("alfred","en")

In [ ]:
print ("Alf %s asd %s" %("ret","wqe"))

In [ ]:
import time
for i in range(10):
    print i
    time.sleep(2)

In [ ]:
from bingSearch import BingSearch
b = BingSearch('FM1eRnjFYoltkKdRDKcwiJMk9VFn9p/yRgru20R5xc4')
print b.bing_search("alfred","Image")
